In [1]:
from ucimlrepo import fetch_ucirepo 
from sklearn.model_selection import train_test_split
import pandas
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# CARREGANDO DADOS

# importando dataset
dataset = fetch_ucirepo(id=320) # 649 registros e 30 atributos
  
# coletando as informações
data_frame = dataset.data.original

In [3]:
# TRATANDO DADOS

print("PRÉ TRATAMENTO: ", len(data_frame))

# removendo colunas com muitos nulos
tolerancia = len(data_frame) * 0.7
data_frame = data_frame.dropna(axis=1, thresh=tolerancia)

# removendo duplicados
print("VALORES DUPLICADOS: ", data_frame.duplicated().sum())
data_frame = data_frame.drop_duplicates()

# convertendo colunas categóricas em valores inteiros
conversores = {}
for coluna in data_frame.columns:
    if (data_frame[coluna].dtype == type(object)):
        conversor = LabelEncoder()
        data_frame[coluna] = conversor.fit_transform(data_frame[coluna])
        conversores[coluna] = conversor

# convertendo colunas numéricas na escala de 0 a 1
escalador = MinMaxScaler()
data_frame[data_frame.select_dtypes(include="number").columns] = escalador.fit_transform(data_frame.select_dtypes(include="number"))

print("PÓS TRATAMENTO: ", len(data_frame))

PRÉ TRATAMENTO:  649
VALORES DUPLICADOS:  0
PÓS TRATAMENTO:  649


In [4]:
print(data_frame.columns)

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')


In [5]:
# DIVIDINDO O DATASET TRATADO

atributos = data_frame.drop(["G1", "G2", "G3"], axis=1)
respostas = data_frame[["G1", "G2", "G3"]]

a_treino, a_teste, r_treino, r_teste = train_test_split(atributos, respostas, test_size=0.3, random_state=42)


In [6]:
# CRIANDO PARÂMETROS PARA OS MODELOS

lista_parametros = [
    {"id": "1", "criterion": "friedman_mse", "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 5},
    {"id": "2", "criterion": "squared_error", "max_depth": 5, "min_samples_split": 10, "min_samples_leaf": 5},
    {"id": "3", "criterion": "poisson", "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 5},
    {"id": "4", "criterion": "absolute_error", "max_depth": 5, "min_samples_split": 10, "min_samples_leaf": 5},
]

In [7]:
# APLICANDO MODELO

# criando a lista de resultados finais
lista_resultados = []

for parametros in lista_parametros:
    # criando regressor
    regressor = tree.DecisionTreeRegressor(
        criterion=parametros["criterion"],
        max_depth=parametros["max_depth"],
        min_samples_split=parametros["min_samples_split"],
        min_samples_leaf=parametros["min_samples_leaf"],
    )

    # treinando o modelo
    regressor.fit(a_treino, r_treino)

    # prevendo respostas
    r_previsao = regressor.predict(a_teste)

    # calculando métricas
    mse = mean_squared_error(r_teste, r_previsao)
    mae = mean_absolute_error(r_teste, r_previsao)
    r2 = r2_score(r_teste, r_previsao)

    # salvando resultados
    lista_resultados.append([parametros["id"], mse, mae, r2])

In [8]:
# EXIBINDO RESULTADOS
tabela_resultados = pandas.DataFrame(lista_resultados, columns=["Identificador", "ErroQuadráticoMédio", "ErroAbsolutoMédio", "R2"])
print(tabela_resultados)

  Identificador  ErroQuadráticoMédio  ErroAbsolutoMédio        R2
0             1             0.031360           0.135080 -0.158811
1             2             0.023462           0.118802  0.127015
2             3             0.031141           0.134533 -0.151533
3             4             0.022792           0.109897  0.155218
